In [1]:
from torchvision import models, datasets
from PIL import Image
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
# dir(models)

In [3]:
# custom class to return image path in the iterator
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

# # EXAMPLE USAGE:
# # instantiate the dataset and dataloader
# data_dir = "your/data_dir/here"
# dataset = ImageFolderWithPaths(data_dir) # our custom dataset
# dataloader = torch.utils.DataLoader(dataset)

# # iterate over data
# for inputs, labels, paths in dataloader:
#     # use the above variables freely
#     print(inputs, labels, paths)

In [4]:
from torchvision import transforms
transform = transforms.Compose([            
 transforms.Resize(299),                                    
 transforms.ToTensor(),                     
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])

In [5]:
import json

with open("../data/imagenet1000.txt") as f:
    inception_classes = eval(f.read())

# multi-image classification

In [6]:
datadir = '../data'

In [7]:
test_data = ImageFolderWithPaths(datadir,transform=transform)
test_data_loader = DataLoader(test_data)

In [23]:
inception = models.inception_v3(pretrained=True)
inception.eval()
print("model ready to predict")

model ready to predict


In [12]:
outputs = []
for data in tqdm(test_data_loader):
        inputs, label, path = data        
        outputs.append((inception(inputs),label, path))

100%|██████████| 5/5 [00:00<00:00,  6.67it/s]


In [14]:
for result in outputs:
    out, label, path = result
    _, index = torch.max(out, 1)
 
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100

    print("class id: ",index[0].item(), inception_classes[index[0].item()], percentage[index[0].item()].item())

class id:  394 sturgeon 21.342702865600586
class id:  532 dining table, board 76.04572296142578
class id:  665 moped 16.894296646118164
class id:  670 motor scooter, scooter 36.5108642578125
class id:  665 moped 20.387617111206055


# single image classification

In [16]:
image_path = "../data/1/kawasaki.jpg"
img = Image.open(image_path)

In [17]:
img_t = transform(img)

In [18]:
inception = models.inception_v3(pretrained=True)
inception.eval()
batch_t = torch.unsqueeze(img_t, 0)
out = inception(batch_t)
print(out.shape)

torch.Size([1, 1000])


In [20]:
_, index = torch.max(out, 1)
 
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
 
print(inception_classes[index[0].item()], percentage[index[0].item()].item())

moped 20.387617111206055


In [22]:
_, indices = torch.sort(out, descending=True)
[(inception_classes[idx.item()], percentage[idx.item()].item()) for idx in indices[0][:5]]

[('moped', 20.387617111206055),
 ('disk brake, disc brake', 5.139692306518555),
 ('motor scooter, scooter', 4.513574600219727),
 ('crash helmet', 0.9927563667297363),
 ('mountain bike, all-terrain bike, off-roader', 0.7166666984558105)]